In [1]:
import os
import numpy as np
from PIL import Image
import math
import time
import cv2 as cv
import tqdm

In [2]:
testing_fg_root = '/home/zzl/dataset/Combined_Dataset/Test_set/test_fg_names.txt'
testing_bg_root = '/home/zzl/dataset/Combined_Dataset/Test_set/test_bg_names.txt'

In [3]:
ADOBE_ROOT = '/home/zzl/dataset/Combined_Dataset/Test_set/Adobe-licensed_images/'

In [5]:
BG_ROOT = '/home/zzl/dataset/VOCtrainval_11-May-2012/VOCdevkit/VOC2012/JPEGImages/'

In [6]:
out_path = '/home/zzl/dataset/Combined_Dataset/Test_set/Adobe-licensed_images/input/'

In [7]:
def composite4(fg, bg, a, w, h):
    fg = np.array(fg, np.float32)
    bg = np.array(bg[0:h, 0:w], np.float32)
    alpha = np.zeros((h, w, 1), np.float32)
    alpha[:, :, 0] = a / 255.
    comp = alpha * fg + (1 - alpha) * bg
    comp = comp.astype(np.uint8)
    return comp


In [8]:
fg_files = [name for name in open(testing_fg_root).read().splitlines()]
bg_files = [name for name in open(testing_bg_root).read().splitlines()]

In [9]:
num_bgs=20

In [10]:
bg_files = os.listdir(BG_ROOT)

In [11]:
bg_len = len(bg_files)

In [12]:

for k, im_name in tqdm.tqdm(enumerate(fg_files)):

    im = cv.imread(os.path.join(ADOBE_ROOT, 'fg',im_name))
    a = cv.imread(os.path.join(ADOBE_ROOT, 'alpha', im_name), 0)


    h, w = im.shape[:2]

    bcount = 0
    for i in range(num_bgs):

        index = np.random.randint(bg_len)
        bg_name = bg_files[index]

        bg = cv.imread(os.path.join(BG_ROOT, bg_name))
        bh, bw = bg.shape[:2]
        wratio = float(w) / float(bw)
        hratio = float(h) / float(bh)
        ratio = wratio if wratio > hratio else hratio     
        if ratio > 1:        
            bg = cv.resize(src=bg, dsize=(math.ceil(bw * ratio), math.ceil(bh * ratio)), interpolation=cv.INTER_CUBIC)

        out = composite4(im, bg, a, w, h)   
        filename = out_path + im_name[:len(im_name)-4] + '_' + str(bcount) + '.png'

        cv.imwrite(filename, out, [cv.IMWRITE_PNG_COMPRESSION, 9])        

        bcount += 1
            # print(k*num_bgs + bcount)

50it [1:02:34, 71.08s/it]
